<a href="https://colab.research.google.com/github/meashu31/AIML-Capstone-CV9/blob/Vadhi_master/Unet_Models_Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Gogle Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
pip install segmentation-models

In [ ]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import data_utils
import numpy as np
from PIL import Image
from keras.utils.layer_utils import count_params
from tabulate import tabulate
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pandas as pd
from IPython.display import clear_output
import calendar
import time
import datetime
import matplotlib.pyplot as plt

from keras.layers import *
from keras.models import *

In [ ]:
path = '/content/drive/My Drive/Capstone/Model_Weights/'
os.chdir(path) #changing the current directory path to the project direcotry where the dataset is available

im_path = '/content/drive/My Drive/Capstone/data/images/'
mask_path = '/content/drive/My Drive/Capstone/data/masks/'
train_list = os.listdir(im_path)
mask_list = os.listdir(mask_path)
print(len(train_list))
print(len(mask_list))

FileNotFoundError: ignored

In [ ]:
X_train, X_val = train_test_split(train_list, test_size=0.20, random_state=53)
print('Length of training data:',len(X_train),'\nLength of validation data:',len(X_val))

Length of training data: 21347 
Length of validation data: 5337


In [ ]:
class DataGenerator(data_utils.Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, list_IDs, image_path, mask_path,preprocessinput, batch_size=32, dim=(1024, 1024),
                 n_channels=1, shuffle=True,resize=False):
        """Initialization
        self.list_IDs = List of all unique patient Ids
        self.image_path = path to image folder
        self.mask_path = path to mask folder
        self.batch_size = batch_size
        self.dim = dimenstion for each input image
        self.n_channels = no.of channels per image
        self.resize = if true, will resize the input image to self.dim
        self.shuffle = True to shuffle label indexes after every epoch
        """
        self.list_IDs = list_IDs
        self.TempList = list_IDs
        self.image_path = image_path
        self.mask_path = mask_path
        self.PreProcess = preprocessinput
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.resize = resize
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        if len(list_IDs_temp)==0:
          print('Trying to access an empty batch. No image left to iterate over. Try changing the batch index')
          return None
        else:
          # Generate data
          X, y = self.__pixel_generation(list_IDs_temp)

          return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __pixel_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim)
        ''' Initialization
         *self.dim is a variable length parameter. It can vary based on the size of image'''
        
        x_image = np.empty((self.batch_size, self.dim[0],self.dim[1],3))
        x_channels = np.empty((self.dim[0],self.dim[1],3))
        y_mask = np.empty((self.batch_size, *self.dim))
        ID_List = []
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            if self.resize == True:
              x_frame = Image.open(self.image_path + ID).resize((self.dim[0],self.dim[1]),Image.ANTIALIAS)
              y_frame = Image.open(self.mask_path + ID).resize((self.dim[0],self.dim[1]),Image.ANTIALIAS)
            else:
              x_frame = Image.open(self.image_path + ID)
              y_frame = Image.open(self.mask_path + ID)
            #print(np.array(x_frame).dtype)
            # Store images
            for z in range(3):
              x_channels[:,:,z] = np.array(x_frame)
            
            x_image[i,] = self.PreProcess(np.array(x_channels, dtype=np.float32))
            # Store masks
            temp = np.array(y_frame)
            temp = np.where(temp > 0 , 1, y_frame)           
            y_mask[i,] = temp
            # Store ID
        return x_image, y_mask

## Backboned-UNet

In [ ]:
def create_model(Backbone='vgg16',pretrained=True,ShowSummery=False,TrainModel=True,TestModel=True):
  # preprocess input
  PreprocessInput = get_preprocessing(Backbone)
  x_train = DataGenerator(list_IDs=X_train , image_path = im_path, mask_path=mask_path,dim=(224,224),batch_size=8,resize=True, preprocessinput=PreprocessInput)
  x_val = DataGenerator(list_IDs=X_val , image_path = im_path, mask_path=mask_path,dim=(224,224),batch_size=8,resize=True, preprocessinput=PreprocessInput)

  # define model
  model = Unet(Backbone, encoder_weights='imagenet', input_shape=(224, 224, 3))
  ##-------------------getLast layer and add reshape----------------------
  LastLayerIdx = len(model.layers) - 1
  LastLayerName = model.layers[LastLayerIdx].name
  for layer in model.layers:
    layer.trainable = not pretrained
    LastLayer = model.get_layer(LastLayerName).output
    x = Reshape((224, 224))(LastLayer)

  model = Model(inputs=model.input, outputs=x)
  # Model Overview
  trainable_count = count_params(model.trainable_weights)
  non_trainable_count = count_params(model.non_trainable_weights)

  print(tabulate([['Model', 'UNet'],\
                          ['BackBone', Backbone],\
                          ['Number of Layers', len(model.layers)],\
                          ['Total params', model.count_params()],\
                          ['Trainable params', trainable_count],\
                          ['Non-trainable params', non_trainable_count]],\
                          headers=['Header', 'Value'], tablefmt='orgtbl'))
  print()
  print()
  #If Choosen Show Summery
  if ShowSummery: model.summary()
  if TrainModel:
    # Compile model
    model.compile('Adam', loss=bce_jaccard_loss, metrics=['accuracy',iou_score])
    # checkpoint
    ts = calendar.timegm(time.gmtime())
    readable = datetime.datetime.fromtimestamp(ts).isoformat()
    filepath = path + Backbone + "/model-"+ readable + ".h5"
    checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, save_weights_only=True, mode="min", period=1)
    stop = EarlyStopping(monitor="loss", patience=5, mode="min")
    batch_size = 8
    model.fit(x_train,
              epochs=5,
              steps_per_epoch = len(x_train)/batch_size,
              validation_data=x_val,
              validation_steps = len(x_train)/batch_size*2,
              callbacks= [checkpoint, stop])
  if TestModel:
    test_list = ['0a2c130c-c536-4651-836d-95d07e9a89cf.png','0a03fbf6-3c9a-4e2e-89ce-c7629ae43a27.png']
    test_gen = DataGenerator(list_IDs=test_list , image_path = im_path, mask_path=mask_path,dim=(224,224),batch_size=8,resize=True, preprocessinput=PreprocessInput)
    model.compile('Adam', loss=bce_jaccard_loss, metrics=['accuracy',iou_score])
    n = 0
    sample_image = test_gen[n][0][0]
    sample_image_redim = np.expand_dims(sample_image, axis=0)
    region = model.predict(sample_image_redim)
    f, ((ax1, ax2, ax3)) = plt.subplots(1, 3, figsize = (15, 8))
    ax1.imshow(sample_image[:,:,0], cmap = plt.cm.bone)
    ax1.set_title('Input Image')
    ax1.axis('off')
    ax2.imshow(test_gen[n][1][0], cmap = plt.cm.bone)
    ax2.set_title('Ground Truth Mask')
    ax2.axis('off')
    ax3.imshow(region[0], cmap = plt.cm.bone)
    ax3.set_title('Predicted Mask (no threshold applied)')
    ax3.axis('off')
    plt.show()

In [ ]:
print(tabulate([['UNet', 'VGG-16', 67, 23752273],\
                ['UNet', 'VGG-19', 70, 29061969],\
                ['UNet', 'ResNet-18', 128, 14340570],\
                ['UNet', 'ResNet-34', 200, 24456154],\
                ['UNet', 'ResNet-50', 232, 32561114],\
                ['UNet', 'ResNet-101', 419, 51605466],\
                ['UNet', 'ResNet-152', 606, 67295194],\
                ['UNet', 'DenseNet-121', 469, 12144977],\
                ['UNet', 'DenseNet-169', 637, 19519825],\
                ['UNet', 'DenseNet-201', 749, 26378577],\
                ['UNet', 'Inception-V3', 353, 29933105],\
                ['UNet', 'Inception-ResNet-V2', 822, 62061553],\
                ['UNet', 'MobileNet', 129, 8336337]],
                headers=['Model', 'Backbone', 'Number of Layers', 'Total Parameters'], tablefmt='orgtbl',stralign="center", numalign="center"))

|  Model  |      Backbone       |  Number of Layers  |  Total Parameters  |
|---------+---------------------+--------------------+--------------------|
|  UNet   |       VGG-16        |         67         |      23752273      |
|  UNet   |       VGG-19        |         70         |      29061969      |
|  UNet   |      ResNet-18      |        128         |      14340570      |
|  UNet   |      ResNet-34      |        200         |      24456154      |
|  UNet   |      ResNet-50      |        232         |      32561114      |
|  UNet   |     ResNet-101      |        419         |      51605466      |
|  UNet   |     ResNet-152      |        606         |      67295194      |
|  UNet   |    DenseNet-121     |        469         |      12144977      |
|  UNet   |    DenseNet-169     |        637         |      19519825      |
|  UNet   |    DenseNet-201     |        749         |      26378577      |
|  UNet   |    Inception-V3     |        353         |      29933105      |
|  UNet   | 

In [187]:
#------------------------- Load necessary libraries -------------------------#
from ipywidgets import widgets, Layout, Button, Box, Dropdown, Label,Output,interact, interactive_output
#------------------------- Load widgets -------------------------#
BackBoneLabel = Label(value='Select Backbone')
BackboneDrop = Dropdown(options=[('Select', ""),("VGG-16", "vgg16"),("VGG-19", "vgg19"),
                           ("ResNet-18","resnet18"),("ResNet-34","resnet34"),
                           ("ResNet-50","resnet50"),("ResNet-101","resnet101"),
                           ("ResNet-152","resnet152"),("DenseNet-121","densenet121"),
                           ("DenseNet-169","densenet169"),("DenseNet-201","densenet201"),
                           ("Inception V3","inceptionv3"),("Inception ResNet V2","inceptionresnetv2"),
                           ("MobileNet","mobilenet")],value='')
PreTrainLabel = Label(value='Use PreTrained Weights')
PreTrainedDrop = Dropdown(options=[('Yes', True),("No", False)],value=True)
SummeryLabel = Label(value='Show Model Summary')
SummeryDrop = Dropdown(options=[('Yes', True),("No", False)],value=False)
TrainLabel = Label(value='Train Model')
TrainModel = Dropdown(options=[('Yes', True),("No", False)],value=False)
TestLabel = Label(value='Test Model')
TestModel = Dropdown(options=[('Yes', True),("No", False)],value=False)
#------------------------- Function at Button click -------------------------#
def on_button_clicked(b):
  print("BackboneDrop")

button.on_click(on_button_clicked)
#------------------------- Function on Selection -------------------------#
def OnSelection(BackBoneSelection,PreTrainSelection,SummerySelection,TrainSelection,TestSelection):
  PreTrainStr = None
  ShowSummaryStr = None
  TrainModelStr = None
  TestModelStr = None
  print()
  if BackBoneSelection == "":
    print("Please select a Backbone of your choice")
  else:
    PreTrainStr = "Trained" if PreTrainSelection else "UnTrained"
    ShowSummaryStr = "be" if SummerySelection else "not be"
    TrainModelStr = "be" if SummerySelection else "not be"
    TestModelStr = "be" if TestSelection else "not be"
    print("UNet Model with",PreTrainStr,BackBoneSelection,"backbone will be loaded, Model Summary will",ShowSummaryStr,"displayed, will", TrainModelStr, "trained and will",TestModelStr, "tested with new data.")
    print()
    print("If OK with above configuration, Click Generate Model (OR) Choose required configuration from GUI")
    print()
    button = widgets.Button(description="Generate Model",
                            disabled=False,
                            button_style='success',
                            tooltip='Click to generate model')
    display(button)
#------------------------- Make widget -------------------------#
out = interactive_output(OnSelection, {"BackBoneSelection":BackboneDrop, "PreTrainSelection":PreTrainedDrop, "SummerySelection":SummeryDrop, "TrainSelection":TrainModel, "TestSelection":TestModel})
form_item_layout = Layout(display='flex',flex_flow='row',justify_content='space-between')
form_items = [
    Box([BackBoneLabel,BackboneDrop], layout=form_item_layout),
    Box([PreTrainLabel, PreTrainedDrop], layout=form_item_layout),
    Box([SummeryLabel, SummeryDrop], layout=form_item_layout),
    Box([TrainLabel, TrainModel], layout=form_item_layout),
    Box([TestLabel, TestModel], layout=form_item_layout)]
GUI = Box(form_items, layout=Layout(display='flex',flex_flow='column',border='solid 2px',align_items='stretch',width='27%'))

In [188]:
display(GUI, out)

Box(children=(Box(children=(Label(value='Select Backbone'), Dropdown(options=(('Select', ''), ('VGG-16', 'vgg1…

Output()